# Lab 9 - Structured Output - GPT-4o-mini

In [ ]:
from tqdm.notebook import tqdm
from datetime import datetime
from IPython.display import display, HTML
import numpy as np
import pickle
import openai
import json

#Don't include keys like this, use ENV vars!
with open('config.json') as fd:
    conf = json.loads(fd.read())
    gpt = openai.OpenAI(api_key=conf["openai_key"])

In [ ]:
def complete_function(prompt,function):
    resp = gpt.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant in charge of routing tasks based on a user query.",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
        temperature=0,
        tools=[
            {
                "type": "function",
                "function": function
            }
        ]
    )
    
    try:
        #Parse the JSON response
        content = resp.choices[0]
        tool_call = content.message.tool_calls[0]
        arguments_str = tool_call.function.arguments
        return json.loads(arguments_str)

    except (AttributeError, IndexError, TypeError, json.JSONDecodeError) as e:
        print(e)

    #Fallback to returning None if there was an error
    return None

In [ ]:
def get_classification_prompt(query):
    return f"""#Instructions

For an event management SaaS product, natural language queries need to be classified and routed to the appropriate branch. Classify the following user query:"
    
{query}"""

In [ ]:
def get_classification_function():
    return {
        "name": "task_type",
        "description": "Classify the task based on the user query.",
        "strict": True,
        "parameters": {
          "type": "object",
          "properties": {
            "task_type": {
              "type": "string",
              "enum": ["create_event","list_events","create_attendee","list_attendees","other"]
            }
          },
          "required":["task_type"],
          "additionalProperties":False
        }
    }

In [ ]:
def classify(query):
    prompt = get_classification_prompt(query)
    function = get_classification_function()
    classification = complete_function(prompt,function)
    if classification:
        print(f"""{query} => {classification['task_type']}""")
    return classification

In [ ]:
c = classify("What events are coming up?")
c = classify("Book the Ellison Lodge A for December 4th")
c = classify("Jane Doe is coming to the party on 12/4")
c = classify("Pencil in highland park gazebo for june 11th")
c = classify("Leah, Alice, and Fred are all coming on june 11th")
c = classify("Who is coming in June?")
c = classify("What spaces are available?")